# Analyse en Composantes Principales

In [1]:
# chargement des données
import pandas as pd

donnee = pd.read_excel('temperature.xlsx',sheet_name=0,header=0,index_col=0)

#### Statistiques descriptives

In [2]:
# Statistiques sur les variables
display(donnee.describe(include = "all").round(2))

,Jan,Fev,Mars,Avril,Mai,Juin,Juil,Août,Sept,Oct,Nov,Dec
count,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00
mean,3.97,4.83,8.23,10.98,14.43,17.83,19.83,19.57,16.99,12.32,7.93,4.85
std,2.01,1.87,1.53,1.41,1.50,1.79,2.13,2.01,1.85,1.83,1.80,1.96
min,0.40,1.50,5.60,8.90,11.60,14.40,15.60,16.00,14.70,9.50,4.90,1.30
25%,2.40,3.35,7.55,10.00,13.70,17.15,18.90,18.45,15.85,11.30,6.60,3.45
50%,4.70,5.30,7.80,10.70,14.30,17.50,19.40,19.10,16.40,11.60,7.80,5.40
75%,5.55,6.20,9.55,12.20,15.35,19.00,20.90,20.95,18.45,13.55,9.05,6.35
max,7.50,8.50,10.80,13.30,16.80,20.80,23.30,22.80,20.30,16.00,11.50,8.20


In [3]:
# Dimension des données actives
n = donnee.shape[0]  # nombre de lignes
p = donnee.shape[1]  # nombre de colonnes
print(f'n = {n} et p = {p}')

n = 15 et p = 12


In [ ]:
# pairplot
import seaborn as sns

sns.pairplot(donnee,aspect=1)

#### Notion de distance entre individus

$$\text{d}^2(i,l) = \displaystyle \sum_{k=1}^{K}(x_{ik}-x_{lk})^2$$

In [ ]:
# Distance entre villes
import numpy as np

def distance(x,y):
    return np.sum((x-y)**2)

#   Calcul de la distance
rowdist = pd.DataFrame(np.zeros(shape=(n,n),dtype=float),index = donnee.index,
                       columns = donnee.index)

for i in range(n):
    for j in range(i+1,n):
        rowdist.values[i,j] = distance(donnee.values[i,:],donnee.values[j,:])

# Affichage
display(rowdist.round(1))

In [ ]:
# Affichage
display(rowdist.round(1))

In [ ]:
# Visualisation - heatmap
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (10,8))
sns.heatmap(rowdist,xticklabels=rowdist.columns,yticklabels = rowdist.columns,
            cmap = sns.color_palette("Blues",12),linewidths=0.5)
plt.title('Heatmap des distances entre villes')
plt.show()

#### Matrice des corrélations

In [ ]:
# Matrice de corrélation linéaire entre paires de variables
corr = donnee.corr(method = "pearson")
# Affichage
display(corr.round(2))

In [ ]:
# heatmap de la matrice des corrélation
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (10,8))
sns.heatmap(corr,xticklabels=corr.columns,yticklabels = corr.columns,
            vmin = -1, vmax = +1, center = 0,
            cmap = 'RdBu',linewidths=0.5)
plt.title('Heatmap des corrélations croisées entre variables')
plt.show()

#### Centrage des données

In [ ]:
# Centrage des données
def centrage(x):
    # x est un vecteur
    return (x-x.mean())

# Application
Y = donnee.transform(centrage)
# Affichage
display(Y.round(2))

#### Centrage et réduction des données

In [ ]:
# Centrage et réduction des données
def StandardScaler(x):
    # x est vecteur
    return (x - x.mean())/x.std(ddof=0)

# Application
Z = donnee.transform(StandardScaler)
#Affichage
display(Z.round(2))

In [ ]:
Z.corr(method = "pearson").round(2)

#### Distance euclidienne pondérée

In [ ]:
# Calcul de la distance euclidienne pondérée
dist = pd.DataFrame(np.zeros(shape=(n,n),dtype=float),index = donnee.index,
                    columns = donnee.index)
for i in range(n):
    for j in range(i+1,n):
        dist.values[i,j] = distance(Z.values[i,:],Z.values[j,:])

# Visualisation - heatmap
plt.figure(figsize = (10,8))
sns.heatmap(dist,xticklabels=dist.columns,yticklabels = dist.columns,
            cmap = sns.color_palette("Blues",12),linewidths=0.5)
plt.title('Heatmap des distances entre villes')
plt.show()

#### Notion d'inertie

In [ ]:
# Inertie : moyenne des carrés des distances
InK = (1/n**2)*rowdist.sum().sum()
print('Inertie totale : %.2f'%(InK))

In [ ]:
# Moyennes des variables
meanvar = donnee.mean(axis=0)

# Inertie totale
distorig = donnee.apply(lambda x:np.sum((x-meanvar)**2), axis = 1)
InK2 = (1/n)*distorig.sum()
print('Inertie totale : %.2f'%(InK2))

In [ ]:
# Vérification - variance des variables
variancevar = donnee.var(ddof = 0)

# Inertie totale
InK3 = variancevar.sum()
print('Inertie totale : %.2f'%(InK3))

In [ ]:
# Inertie sur données centrées et réduites
print('Inertie totale (donnée centrée réduite) : %.1f'%(Z.var(ddof=0).sum()))

#### Distance des individus à l'origine

In [ ]:
# Information sur les individus
rowdisto = Z.apply(lambda x : np.sum((x)**2),axis=1)
rowweight = np.ones(n)/n
rowinertie = rowweight*rowdisto.values
rowinfos = pd.DataFrame(np.transpose([rowdisto,rowweight, rowinertie]),
                       columns = ["Disto2", "Poids", "Inertie"],
                       index = donnee.index)
display(rowinfos.round(3))

In [ ]:
# Informations sur les variables
rowmean = Z.mean(axis=1)
vardisto = Z.apply(lambda x : np.sum((x-rowmean)**2)/n,axis=0)
varweight = np.ones(p)/p
varinertie = varweight*vardisto.values
varinfos = pd.DataFrame(np.transpose([vardisto,varweight, varinertie]),
                       columns = ["Disto2", "Poids", "Inertie"],
                       index = donnee.columns)
display(varinfos)

In [ ]:
np.sum(varinertie)

### ACP via la diagonalisation de la matrice des corrélations

In [ ]:
# Diagonalisation de la matrice des corrélations
eigenvalue, eigenvector = np.linalg.eig(corr)

In [ ]:
# eigenvalue dataframe
percent = np.array([100*x/sum(eigenvalue) for x in eigenvalue])
cumpercent = np.cumsum(percent)
columns = ['valeur propre','pourcentage d\'inertie',
           'pourcentage d\'inertie cumulée']
index = ['Dim.{}'.format(x+1) for x in range(p)]
Eigen = pd.DataFrame(np.transpose([eigenvalue,percent,cumpercent]),
                     index=index,columns = columns)
Eigen.index.name = 'Dimension'
# Affichage
display(Eigen.round(2))

### ACP via la décomposition en valeurs singulières

Rappellons que la SVD (singular Values Decomposition) ou décomposition en valeurs singulières d'une matrix $X$ de taille $n\times p$ consiste à calculer les valeurs singulières $\lambda = \left(\lambda_{1},\dots,\lambda_{p}\right)$ (triées par ordre décroissant) et les matrices unitaires $U = \left[\mu_{1},\dots,\mu_{n}\right]$ et $V = \left[\nu_{1},\dots,\nu_{p}\right]$ tels que $$X = \displaystyle \sum_{k}^{p}\lambda_{k}\mu_{k}\nu_{k}^{T}=U\text{diag}(\lambda)V^{T}$$

Les colonnes $\nu_{k}$ de $\textbf{V}$ sont $\textbf{les axes principaux}$ et les vecteurs $\lambda_{k}\mu_{k}$ sont les $\textbf{composantes principales}$.

En python, c'est la fonction $\textbf{np.linalg.svd}$ qui effectue la SVD d'une amtrice. Elle renvoie les valeurs singulières $\lambda$ et les matrices unitaires $\textbf{U}$ et $\textbf{V}$.

Pour l'ACP, nous effectuons la SVD du tableau des données centrées.

In [ ]:
# Décomposition en valeurs singulières
U, delta ,V  = np.linalg.svd(Z,full_matrices=False)

In [ ]:
# Correction des delta
lambd = delta**2/n
# Affichage de delta et lambda
display(pd.DataFrame(np.array([delta,lambd]),columns = index,
                   index=['delta','lambda']).round(2))

#### Valeurs propres

In [ ]:
# Visualisation des valeurs propres
def screeplot(data,choice=None,figsize=None):
    #
    p = data.shape[0]
    fig,axes = plt.subplots(figsize = figsize); axes.grid()
    axes.set_xlabel('Dimensions',fontsize=14)
    axes.set_title('Scree plot',fontsize=14)
    axes.set_xticks([x for x in range(1,p+1)])
    if choice is None or choice=='scree plot':
        eigen = data.iloc[:,0].round(2)
        ylim = np.max(eigen)+1
        axes.set_ylim(0,ylim)
        axes.bar(np.arange(1,p+1),eigen.values,width=0.9)
        axes.plot(np.arange(1,p+1),eigen.values,c="black")
        axes.set_ylabel('Eigenvalue',fontsize=13)
        ## Add text
        for i in range(p):
            axes.scatter(i+1,eigen.values[i],color='black',alpha=1)
            axes.text(i+.75,0.10+eigen.values[i],str(eigen.values[i]),
                     color = 'black')
    elif choice == "percentage":
        percent = data.iloc[:,1].round()
        axes.set_ylim(0,100)
        axes.bar(np.arange(1,p+1),percent.values,width=0.9)
        axes.plot(np.arange(1,p+1),percent.values,c="black")
        axes.set_ylabel('Percentage of variance',fontsize=13)
        ## Add text
        for i in range(p):
            axes.scatter(i+1,percent.values[i],color='black',alpha=1)
            axes.text(i+.6,0.10+percent.values[i],f'{percent.values[i]}%',
                     color = 'black',fontweight='bold',fontsize=12)
    elif choice == "cumulative":
        cumul = data.iloc[:,2].round()
        axes.set_ylim(0,105)
        axes.bar(np.arange(1,p+1),cumul.values,width=0.9)
        axes.set_ylabel('Cumulative percentage of variance',fontsize=13)
    plt.show()
# Affichage
screeplot(data=Eigen,figsize=(8,6))

#### Coordonnées des individus

In [ ]:
# coordonnées factorielles des individus
rowcoord = pd.DataFrame(np.dot(Z,eigenvector),index = donnee.index,
                       columns = index)
# Affichage
display(rowcoord.loc[:,['Dim.1','Dim.2']].T.round(2))

In [ ]:
# Fonction de visualisation en 2D
def pca_row_plot(data,eigen,axei,axej,figsize=None):
    try:
        if axei==axej:
            raise ValueError('Erreur: axei doit être différent de axej.')
        elif axei>axej:
            raise ValueError('Erreur: axei doit être inférieur à axej.')
        elif axei<0 or axej<0:
            msg = 'Erreur: les valeurs des axes doivent être positives ou nulles.'
            raise ValueError(msg)
        else:
            # set limite
            n = data.shape[0]
            # Valeurs propres
            percent = np.array([100*x/sum(eigen) for x in eigen])
            dimi = round(percent[axei],2); dimj = round(percent[axej],2)

            # Graphique
            fig, axes = plt.subplots(figsize = figsize); axes.grid()
            axes.axis([-8,8,-6,6])
            axes.set_title("Projection des individus")
            axes.set_xlabel(f"Dim.{1+axei} ({dimi}%)")
            axes.set_ylabel(f"Dim.{1+axej} ({dimj}%)")
            for i in range(n):
                plt.scatter(data.iloc[i,axei], data.iloc[i,axej],
                            c = "black", alpha = 1)
                axes.text(data.iloc[i,axei],data.iloc[i,axej],data.index[i],
                          color = "black", fontsize = 11)
            plt.axhline(0, color='blue',linestyle="--", linewidth=0.5)
            plt.axvline(0, color='blue',linestyle="--", linewidth=0.5)
            plt.show()
            
    # if false then raise the value error
    except ValueError as e:
            print(e) 

# Nuage des individus sur les axes 1 et 2
pca_row_plot(data=rowcoord,
             eigen=eigenvalue,
             axei=0,
             axej=1,
             figsize=(12,8))

#### Distance entre variables

In [ ]:
# Distance en variables
vardist = pd.DataFrame(np.zeros(shape=(p,p),dtype=float),index = donnee.columns,
                       columns = donnee.columns)
for j in range(p):
    for k in range(j+1,p):
        vardist.iloc[j,k] = 2*(1-corr.iloc[j,k])
# Visualisation - heatmap
plt.figure(figsize = (10,8))
sns.heatmap(vardist,xticklabels=dist.columns,yticklabels = dist.columns,
            vmin = 0, vmax = 4,cmap = sns.color_palette("Blues",12),linewidths=0.5)
plt.title('Heatmap des distances entre mois')
plt.show()

#### Coordonnées des variables

In [ ]:
# coordonnées des variables
varcoord = pd.DataFrame(eigenvector*np.sqrt(eigenvalue),columns = index,
                       index = donnee.columns)
varcoord.index.name = 'mois'
# Affichage
display(varcoord.loc[:,['Dim.1','Dim.2']].T.round(2))

In [ ]:
# Visualisation du nuage des variables (=Cercle de corrélation)
def pca_var_plot(data,eigen,axei,axej,figsize=None):
    try:
        if axei==axej:
            raise ValueError('Erreur: axei doit être différent de axej.')
        elif axei>axej:
            raise ValueError('Erreur: axei doit être inférieur à axej.')
        elif axei<0 or axej<0:
            msg = 'Erreur: les valeurs des axes doivent être positives ou nulles.'
            raise ValueError(msg)
        else:
            p = data.shape[0]
            # Valeurs propres
            percent = np.array([100*x/sum(eigen) for x in eigen])
            dimi = round(percent[axei],2); dimj = round(percent[axej],2)
            
            # Graphique
            fig, axes = plt.subplots(figsize = figsize)
            axes.grid()
            axes.axis([-1.5,1.5,-1.5,1.5])
            axes.set_title("Cercle de corrélation")
            axes.set_xlabel(f"Dim.{1+axei} ({dimi}%)")
            axes.set_ylabel(f"Dim.{1+axej} ({dimj}%)")
            for j in range(p):
                axes.arrow(0,0,data.iloc[j,axei],data.iloc[j,axej],
                           head_width = 0.02,head_length = 0.02,color='black')
                axes.text(data.iloc[j,axei],data.iloc[j,axej],data.index[j],
                          color = "black")
            # cercle
            from matplotlib.patches import Ellipse
            ellipse = Ellipse((0,0),width = 2, height = 2,facecolor = "none", 
                              edgecolor = "blue")
            axes.add_patch(ellipse)
            plt.axhline(0, color='blue',linestyle="--", linewidth=0.5)
            plt.axvline(0, color='blue',linestyle="--", linewidth=0.5)
            plt.show()
    # if false then raise the value error
    except ValueError as e:
            print(e)  

# Nuage des variables
pca_var_plot(data=varcoord,
             eigen=eigenvalue,
             axei=0,
             axej=1,
             figsize=(10,10))

In [ ]:
# Cercle des corrélations des variables
from mlxtend.plotting import plot_pca_correlation_graph

fig,corr_mat = plot_pca_correlation_graph(Z.values,Z.columns,
                                                        dimensions=(1, 2),
                                                        figure_axis_size=10)
plt.grid(True)
plt.show()

In [ ]:
display(corr_mat)

In [ ]:
# coordonnées des individus
rowcoord_svd = pd.DataFrame(U * delta.reshape(1, -1),index = donnee.index,
                       columns = index)
# Affichage axe 1 et axe 2
display(rowcoord_svd.loc[:,['Dim.1','Dim.2']].T.round(2))

In [ ]:
# Coordonnées factorielles des variables
varcoord_svd = pd.DataFrame(V.T.dot(np.diag(np.sqrt(lambd))),
                           index = donnee.columns, columns = index)
varcoord_svd.index.name = 'mois'
display(varcoord_svd.loc[:,['Dim.1','Dim.2']].T.round(2))

### Relation de transition

#### Coordonées de l'individu i

In [ ]:
# relation de transition - coordonnées des individus
transition1 = Z.dot(varcoord)/np.sqrt(eigenvalue)
display(transition1.iloc[:,[0,1]].round(2))

#### Coordonnées de la variable k

In [ ]:
# relation de transition - coordonnées des variables
transition2 = Z.T.dot(rowcoord)/(n*np.sqrt(eigenvalue))
display(transition2.iloc[:,[0,1]].T.round(2))

#### Biplot

In [ ]:
# Nuage simultané
def biplot(data1,data2,eigen,axei,axej,figsize=None):
    # Représentation simultanée des indvidus et des variables
    try:
        if axei==axej:
            raise ValueError('Erreur: axei doit être différent de axej.')
        elif axei>axej:
            raise ValueError('Erreur: axei doit être inférieur à axej.')
        elif axei<0 or axej<0:
            msg = 'Erreur: les valeurs des axes doivent être positives ou nulles.'
            raise ValueError(msg)
        else:
            n,p = data1.shape
            # Valeurs propres
            percent = np.array([100*x/sum(eigen) for x in eigen])
            dimi = round(percent[axei],2); dimj = round(percent[axej],2)
            
            # Biplot
            fig = plt.figure(figsize=(12,8))
            axes1 = fig.add_subplot(111)
            axes2 = axes1.twiny()
            axes2 = axes2.twinx()
            axes1.grid()
            axes2.axis([-1.2,1.2,-1.2,1.2])
            axes1.axis([-8,8,-6,6])
            axes1.set_title("Biplot")
            axes1.set_xlabel(f"Dim.{1+axei} ({dimi}%)")
            axes1.set_ylabel(f"Dim.{1+axej} ({dimj}%)")
            # Affichage des individus
            for i in range(n):
                axes1.scatter(data1.iloc[i,0],data1.iloc[i,1],c = "black",
                              alpha = 1)
                axes1.text(data1.iloc[i,0],data1.iloc[i,1],data1.index[i],
                           color = "black",fontsize=12)
            # Affichage des variables
            for k in range(p):
                axes2.arrow(0,0,data2.iloc[k,0],data2.iloc[k,1],
                            head_width = 0.02,head_length = 0.02,
                            color='blue',linestyle='--')
                axes2.text(data2.iloc[k,0],data2.iloc[k,1],data2.index[k],
                           color = "blue",fontsize=12)
            plt.axhline(0, color='blue',linestyle="--", linewidth=0.5)
            plt.axvline(0, color='blue',linestyle="--", linewidth=0.5)
            plt.show()
            
    except ValueError as e:
            print(e)  

# Affichage
biplot(data1=rowcoord,data2=varcoord,eigen=eigenvalue,axei=0,axej=1,
       figsize=(10,10))

## Outils d'aide à l'interprétation

### Analyse du point de vue des individus

#### Contributions factorielles des individus

In [ ]:
# contributions factorielles des individus
rowcontrib = rowcoord.apply(lambda x: 100*x**2/(n*eigenvalue),axis=1)
# Affichage
display(rowcontrib.iloc[:,[0,1]].T.round(2))

In [ ]:
# vérifions la théorie
display(rowcontrib.sum(axis=0))

In [ ]:
# Affichage graphique des contributions
def plot_graph(data,axis,xlabel,title,figsize=None):
    p = data.shape[1]
    try:
        if axis<0 or axis>p:
            raise ValueError(f'axis doit être compris entre {0} et {p-1}.')
        else:
            sort = data.sort_values(by=f'Dim.{1+axis}', ascending=True)
            sort.iloc[:,axis].plot.barh(figsize=figsize)
            plt.xlabel(xlabel)
            plt.title(f'{title} in axis {1+axis}')
            plt.show()
    except ValueError as f:
        print(f)   

# Contribution axe 1
plot_graph(data=rowcontrib,
           axis=0,
           xlabel = 'Contribution (%)',
           title = 'Rows contributions',
           figsize=(10,8))

# Contribution axe 2
plot_graph(data=rowcontrib,
           axis=1,
           xlabel = 'Contribution (%)',
           title = 'Rows contributions',
           figsize=(10,8))

### Cosinus carré des individus

In [ ]:
# Cosinus carré des individus
rowcos2 = rowcoord.apply(lambda x : x**2/rowdisto, axis=0)
#Affichage
display(rowcos2.iloc[:,[0,1]].T.round(2))

In [ ]:
# Cosinus carré axe 1
plot_graph(data=rowcos2,
           axis=0,
           xlabel = 'Cosinus 2',
           title = 'Rows cosinus 2',
           figsize=(10,8))

# Cosinus carré axe 2
plot_graph(data=rowcos2,
           axis=1,
           xlabel = 'Cosinus 2',
           title = 'Rows cosinus 2',
           figsize=(10,8))

### Analyse du point de vue des variables

#### Contributions des variables

In [ ]:
# contributions factorielles des variables
varcontrib = varcoord.apply(lambda x : 100*x**2/eigenvalue,axis=1)
# Affichage
display(varcontrib.iloc[:,[0,1]].T.round(2))

In [ ]:
# Contribution variable axe 1
plot_graph(data=varcontrib,
           axis=0,
           xlabel = 'Contribution (%)',
           title = 'Columns contributions',
           figsize=(10,8))

# Contribution variable axe 2
plot_graph(data=varcontrib,
           axis=1,
           xlabel = 'Contribution (%)',
           title = 'Columns contributions',
           figsize=(10,8))

#### Cosinus carré des variables

In [ ]:
# Cosinus carré des variables
varcos2 = varcoord.apply(lambda x : x**2)
# Affichage
display(varcos2.iloc[:,[0,1]].T.round(2))

In [ ]:
# Cosinus carré variable axe 1
plot_graph(data=varcos2,
           axis=0,
           xlabel = 'Cosinus 2',
           title = 'Columns cosinus 2',
           figsize=(10,8))

# Cosinus carré variable axe 2
plot_graph(data=varcos2,
           axis=1,
           xlabel = 'Cosinus 2',
           title = 'Columns cosinus 2',
           figsize=(10,8))

## Elements supplémentaires

### Individus supplémentaires

In [ ]:
# individus supplémentaires
ind_sup = pd.read_excel('temperature.xlsx',sheet_name=3,header=0,index_col=0)

In [ ]:
# statistiques descriptives
display(ind_sup.describe(include='all').round(2))

In [ ]:
# dimensions
m,p = ind_sup.shape
print(f'm = {m} et p = {p}')

In [ ]:
# Normalisation des données supplémentaires
A = ind_sup.apply(lambda x : (x-donnee.mean(axis=0))/donnee.std(ddof=0,axis=0),
                   axis=1)
display(A.round(2))

In [ ]:
# coordonnées factorielles des villes supplémentaires
rowsupcoord = pd.DataFrame(np.zeros(shape=(m,p),dtype=float),columns = index,
                           index = ind_sup.index)
for j in range(p):
    rowsupcoord.iloc[:,j] = 1/np.sqrt(eigenvalue[j])*A.dot(varcoord.iloc[:,j])

# Affichage
rowsupcoord.index.name = 'villes'
display(rowsupcoord.iloc[:,[0,1]].T.round(2))

In [ ]:
# Fonction de visualisation en 2D
def pca_rowsup_plot(data1,data2,eigen,axei,axej,figsize=None):
    try:
        if axei==axej:
            raise ValueError('Erreur: axei doit être différent de axej.')
        elif axei>axej:
            raise ValueError('Erreur: axei doit être inférieur à axej.')
        elif axei<0 or axej<0:
            msg = 'Erreur: les valeurs des axes doivent être positives ou nulles.'
            raise ValueError(msg)
        else:
            n = data1.shape[0]
            m = data2.shape[0]
            # Valeurs propres
            percent = np.array([100*x/sum(eigen) for x in eigen])
            dimi = round(percent[axei],2); dimj = round(percent[axej],2)

            # Graphique
            fig, axes = plt.subplots(figsize = figsize); axes.grid()
            axes.axis([-15,15,-8,8])
            axes.set_title("Projection des individus")
            axes.set_xlabel(f"Dim.{1+axei} ({dimi}%)")
            axes.set_ylabel(f"Dim.{1+axej} ({dimj}%)")
            # Positionnement des individus actifs
            for i in range(n):
                plt.scatter(data1.iloc[i,axei], data1.iloc[i,axej],
                            c = "black", alpha = 1)
                axes.text(data1.iloc[i,axei],data1.iloc[i,axej],data1.index[i],
                          color = "black", fontsize = 11)
            # positionnement des individus illustratifs
            for j in range(m):
                plt.scatter(data2.iloc[j,axei], data2.iloc[j,axej],
                            c = "blue", alpha = 1)
                axes.text(data2.iloc[j,axei],data2.iloc[j,axej],data2.index[j],
                          color = "blue", fontsize = 11)
            plt.axhline(0, color='blue',linestyle="--", linewidth=0.5)
            plt.axvline(0, color='blue',linestyle="--", linewidth=0.5)
            plt.show()
            
    # if false then raise the value error
    except ValueError as e:
            print(e)    

# Nuage des individus
pca_rowsup_plot(data1 = rowcoord,
                data2 = rowsupcoord,
                eigen = eigenvalue,
                axei = 0,
                axej = 1,
                figsize=(12,8))

#### Variables supplémentaires quantitatives

In [ ]:
# variables supplémentaires quantitatives
vsQuant = pd.read_excel('temperature.xlsx',sheet_name=1,header=0,index_col=0)

In [ ]:
# Dimensions des variables supplémentaires
n, q = vsQuant.shape
print(f'm = {n} et q = {q}')

In [ ]:
# corrélation avec les axes factorielles
varsupcoord = pd.DataFrame(np.zeros(shape=(q,p),dtype=float),columns = index,
                           index = vsQuant.columns)
for k in range(q):
    for l in range(p):
        varsupcoord.iloc[k,l] = np.corrcoef(vsQuant.iloc[:,k],
                                            rowcoord.iloc[:,l])[1,0]

# Affichage
display(varsupcoord.iloc[:,[0,1]].T.round(2))

In [ ]:
# Visualisation du nuage des variables (=Cercle de corrélation)
def pca_varsup_plot(data1,data2,eigen,axei,axej,figsize=None):
    try:
        if axei==axej:
            raise ValueError('Erreur: axei doit être différent de axej.')
        elif axei>axej:
            raise ValueError('Erreur: axei doit être inférieur à axej.')
        elif axei<0 or axej<0:
            msg = 'Erreur: les valeurs des axes doivent être positives ou nulles.'
            raise ValueError(msg)
        else:
            p = data1.shape[0]
            q = data2.shape[0]
            # Valeurs propres
            percent = np.array([100*x/sum(eigen) for x in eigen])
            dimi = round(percent[axei],2); dimj = round(percent[axej],2)
            
            # Graphique
            fig, axes = plt.subplots(figsize = figsize)
            axes.grid()
            axes.axis([-1.5,1.5,-1.5,1.5])
            axes.set_title("Cercle de corrélation")
            axes.set_xlabel(f"Dim.{1+axei} ({dimi}%)")
            axes.set_ylabel(f"Dim.{1+axej} ({dimj}%)")
            for j in range(p):
                axes.arrow(0,0,data1.iloc[j,axei],data1.iloc[j,axej],
                           head_width = 0.02,head_length = 0.02,color='black')
                axes.text(data1.iloc[j,axei],data1.iloc[j,axej],data1.index[j],
                          color = "black")
            for k in range(q):
                axes.arrow(0,0,data2.iloc[k,axei],data2.iloc[k,axej],
                           head_width = 0.02, head_length = 0.02,
                           color='blue',linestyle='--')
                axes.text(data2.iloc[k,axei],data2.iloc[k,axej],data2.index[k],
                          color = "blue")
            # cercle
            from matplotlib.patches import Ellipse
            ellipse = Ellipse((0,0),width = 2, height = 2, facecolor = "none", 
                              edgecolor = "blue")
            axes.add_patch(ellipse)
            plt.axhline(0, color='blue',linestyle="--", linewidth=0.5)
            plt.axvline(0, color='blue',linestyle="--", linewidth=0.5)
            plt.show()
    # if false then raise the value error
    except ValueError as e:
            print(e)  

# Nuage des variables
pca_varsup_plot(data1 =varcoord,
                data2 = varsupcoord,
                eigen=eigenvalue,
                axei=0,
                axej=1,
                figsize=(10,10))

#### variables qualitatives supplémentaires

In [ ]:
# variables supplémentaires qualitatives
vsQual = pd.read_excel('temperature.xlsx',sheet_name=2,header=0,index_col=0)

In [ ]:
# Dimensions des variables supplémentaires
n, r = vsQual.shape
print(f'm = {n} et r = {r}')

In [ ]:
# Modalités de la variable
modalites = np.unique(vsQual)
print('modalite :',modalites)

In [ ]:
# Fonction de visualisation en 2D
def pca_rowqual_plot(data,vsqual,eigen,axei,axej,figsize=None):
    try:
        if axei==axej:
            raise ValueError('Erreur: axei doit être différent de axej.')
        elif axei>axej:
            raise ValueError('Erreur: axei doit être inférieur à axej.')
        elif axei<0 or axej<0:
            msg = 'Erreur: les valeurs des axes doivent être positives ou nulles.'
            raise ValueError(msg)
        else:
            # set limite
            n = data.shape[0]
            # Valeurs propres
            percent = np.array([100*x/sum(eigen) for x in eigen])
            dimi = round(percent[axei],2); dimj = round(percent[axej],2)

            # Graphique
            fig, axes = plt.subplots(figsize = figsize); axes.grid()
            axes.axis([-8,8,-6,6])
            axes.set_title("Projection des individus")
            axes.set_xlabel(f"Dim.{1+axei} ({dimi}%)")
            axes.set_ylabel(f"Dim.{1+axej} ({dimj}%)")
            cdict = {'A': 'green','B': 'blue', 'C':'red'}
            for group in np.unique(vsqual):
                idx = np.where(vsqual==group)
                axes.scatter(data.iloc[:,axei].values[idx[0]],
                             data.iloc[:,axej].values[idx[0]],
                             label = group, c = cdict[group])
                for i in idx[0]:
                    axes.text(data.iloc[:,0].values[i],data.iloc[:,1].values[i],
                              s=data.index[i],c =cdict[group],fontsize = 11)
            axes.legend()
            plt.axhline(0, color='blue',linestyle="--", linewidth=0.5)
            plt.axvline(0, color='blue',linestyle="--", linewidth=0.5)
            plt.show()
            
    # if false then raise the value error
    except ValueError as e:
            print(e) 

# Nuage des individus sur les axes 1 et 2
pca_rowqual_plot(data=rowcoord,vsqual = vsQual,eigen=eigenvalue,
                 axei=0,axej=1,figsize=(12,8))

In [ ]:
# Barycentre des modalités
df = pd.concat([rowcoord,vsQual],axis=1)
# Moyennes conditionnelles
condmean = df.pivot_table(index = 'groupe',values = ['Dim.1','Dim.2'],
                          aggfunc = pd.Series.mean)
display(condmean.T.round(2))

In [ ]:
# Fonction de visualisation en 2D
def pca_rowsupqual_plot(data1,data2,data3,eigen,axei,axej,figsize=None):
    try:
        if axei==axej:
            raise ValueError('Erreur: axei doit être différent de axej.')
        elif axei>axej:
            raise ValueError('Erreur: axei doit être inférieur à axej.')
        elif axei<0 or axej<0:
            msg = 'Erreur: les valeurs des axes doivent être positives ou nulles.'
            raise ValueError(msg)
        else:
            n = data1.shape[0];m = data2.shape[0];r = data3.shape[0]
            # Valeurs propres
            percent = np.array([100*x/sum(eigen) for x in eigen])
            dimi = round(percent[axei],2); dimj = round(percent[axej],2)
            

            # Graphique
            fig, axes = plt.subplots(figsize = figsize); axes.grid()
            axes.axis([-15,15,-8,8])
            axes.set_title("Projection des individus")
            axes.set_xlabel(f"Dim.{1+axei} ({dimi}%)")
            axes.set_ylabel(f"Dim.{1+axej} ({dimj}%)")
            # Positionnement des individus actifs
            for i in range(n):
                plt.scatter(data1.iloc[i,axei], data1.iloc[i,axej],
                            c = "black", alpha = 1)
                axes.text(data1.iloc[i,axei],data1.iloc[i,axej],data1.index[i],
                          color = "black", fontsize = 11)
            # positionnement des individus illustratifs
            for j in range(m):
                plt.scatter(data2.iloc[j,axei], data2.iloc[j,axej],
                            c = "blue", alpha = 1)
                axes.text(data2.iloc[j,axei],data2.iloc[j,axej],data2.index[j],
                          color = "blue", fontsize = 11)
            # positionnement des modalités
            for k in range(r):
                plt.scatter(data3.iloc[k,axei], data3.iloc[k,axej],
                            c = "red", alpha = 1)
                axes.text(data3.iloc[k,axei],data3.iloc[k,axej],data3.index[k],
                          color = "red", fontsize = 12)
            plt.axhline(0, color='blue',linestyle="--", linewidth=0.5)
            plt.axvline(0, color='blue',linestyle="--", linewidth=0.5)
            plt.show()
            
    # if false then raise the value error
    except ValueError as e:
            print(e)    

# Nuage des individus
pca_rowsupqual_plot(data1 = rowcoord,
                    data2 = rowsupcoord,
                    data3 = condmean,
                    eigen = eigenvalue,
                    axei = 0,
                    axej = 1,
                    figsize=(12,8))

## Description des classes

#### Description des classes par les variables explicatives quantitatives

$$\text{Corr}(k,F_\alpha)$$

In [ ]:
#Description
from scipy.stats import pearsonr

descAxe1 = pd.DataFrame(np.zeros(shape=(p,2),dtype=float),index = donnee.columns,
                        columns = ['correlation','p.value'])

for i in range(p):
    descAxe1.iloc[i,0],descAxe1.iloc[i,1] = pearsonr(donnee.iloc[:,i],
                                                     rowcoord.iloc[:,0])

# Affichage
display(descAxe1.sort_values(by='correlation',ascending=False).round(4))

In [ ]:
pd.concat([rowinfos,rowcoord[['Dim.1','Dim.2','Dim.3']],rowcontrib[['Dim.1','Dim.2','Dim.3']],
           rowcos2[['Dim.1','Dim.2','Dim.3']]],axis=1).round(3)

In [ ]:
pd.concat([varcoord[['Dim.1','Dim.2','Dim.3']],varcontrib[['Dim.1','Dim.2','Dim.3']],
           varcos2[['Dim.1','Dim.2','Dim.3']]],axis=1).round(3)

In [ ]:
# Exporter les coordonnées factorielles
#rowcoord.to_pickle("temperature.pkl")  